In [127]:
import numpy as np
import os
import pandas as pd
from pandas_ods_reader import read_ods

from copy import deepcopy

data_path = 'equation_genetic.csv'
VARIANT = 0

In [279]:
class Function:
    def __init__(self, variant, path, absolute = True):
        self.variant = variant
        self.path = path
        self.func_str = self._get_function_str()
        self.absolute = absolute
        
        self.variables = [v for v in list(set(self.func_str)) if v.isalpha() ]
#         self.evaluate = np.vectorize(self.evaluate)

    def _get_function_str(self):
        f_str = "u^powU * w^powW * x^powX * y^powY * z^powZ + u^powU1 * w^powW1 * x^powX1 * y^powY1 * z^powZ1 + u^powU2 * w^powW2 * x^powX2 * y^powY2 * z^powZ2 + u^powU3 * w^powW3 * x^powX3 * y^powY3 * z^powZ3 + u^powU4 * w^powW4 * x^powX4 * y^powY4 * z^powZ4 = Result"
        df = pd.read_csv(self.path)
        self.data = df
        variables = list(df.columns.values)

        for var in variables:
            key = var.replace(" ", "")
            key = key[0].lower() + key[1:] if key[0] is not 'R' else key
            value = df[var][self.variant]
            f_str = re.sub(r"\b{}\b".format(key),str(value) ,f_str)

        f_str = f_str.replace('^', '**')
        f_str = f_str.replace(' ', '')
        
        sign = f_str[ f_str.index('=') + 1 ]
        replace_to = ['=-', '+'] if sign is '-' else ['=', '-'] 
        f_str = f_str.replace(replace_to[0], replace_to[1])
        
        
        return f_str
    
    def get_pairs(self, features):
        result = {}
        for value, var_name in zip(features, self.variables):
            result[var_name] = value
        return result
    
    
    def evaluate(self, features):
        f = self.func_str
        """
            params: dict
                u, w, x, y, z
        """

        if len(features.shape) == 1:
            for value, var_name in zip(features, self.variables):
                f = f.replace(var_name, str(value))
            y = eval(f)
            y = abs(y) if self.absolute else y
            return eval(f)  
        
        elif len(features.shape) == 2:
            out = []
            for line in features:
                f = self.func_str
                for value, var_name in zip(line, self.variables):
#                     print(f, value, var_name)
                    f = f.replace(var_name, str(value))
                y = eval(f)
                y = abs(y) if self.absolute else y
                out.append(y)  
            return np.array(out)
        else:
            raise('Something is going wrong :(')
        
        

    
    

In [401]:
from joblib import Parallel, delayed
from math import sqrt


def sort_a_by_b(A,B):
    return np.array([a for _,a in sorted(zip(B,A))])

class Genetic:
    def __init__(self, n_features, population_size, init_bounds, epochs = 10_000):
        self.epochs = epochs
        self.n_features = n_features
        self.X = np.random.uniform(*init_bounds, [population_size, n_features])
        print(f'X.shape: {self.X.shape}')
    
    def fit(self, func_to_optimize, selection = "Turnir", select_rate = .5, single_crossing = True,\
           p_mutation = .05):
        self.select_rate = select_rate
        self.func = func_to_optimize
        self.single_crossing = single_crossing
        self.p_mutation = p_mutation
        
        X = self.X
        
        self._fit(X)
        
    def _fit(self, X):
        for _ in range(self.epochs):
            X = self._select(X)
            X = self._cross(X)
            X = self._mutation(X)

        
    def _select(self, X):
        selected_num = int(X.shape[0] * self.select_rate)
        y = self.func.evaluate(X)
        idxs_X = np.arange(X.shape[0])
        idxs_X_sorted = sort_a_by_b(idxs_X, y)
        
        X = X[idxs_X_sorted]
        X = X[:selected_num]
        return X[:]
    
    def _cross(self, X):
        """
        Одноточечная (в третьем гене) + многоточечная (во втором и четвертом гене)
        """
        n = X.shape[0]
        pairs = [(p1, p2) for p1 in np.arange(n) for p2 in np.arange(p1+1,n)]
        
        children  = []
        if self.single_crossing:
            switch = 3 - 1
            for i, j in pairs:
                x1 = np.append(X[i][:switch], X[j][switch:])
                x2 = np.append(X[i][switch:], X[j][:switch])
                children.append(x1)
                children.append(x2)
                
        return np.array(children)
    
    def _mutation(self, X):
        """Каждый бит всех потомков мутирует с некоторой вероятностью p"""
        def _irradiate(x):
            x_ = np.empty_like(x)
            for i in range(self.n_features):
                k =  np.random.uniform()
                if self.p_mutation >= k:
                    x_[i] = np.random.uniform(-2000, 2000)
                else:
                    x_[i] = x[i]
            return x_
        
        print(np.sum(X))
        new_X = []
        for x in X:
            x_ = _irradiate(x)
            new_X.append(x_)
        new_X = np.array(new_X)
        return new_X
        
    
        
        

In [402]:
box = Genetic(n_features=5, population_size= 100, init_bounds=[-100, 100], epochs=1)

X.shape: (100, 5)


In [403]:
box.fit(func, p_mutation=.5)

-43372.3405185438
-43372.3405185438 75192.66221146466


In [289]:
a = np.arange(5)
for i, j in [(p1, p2) for p1 in np.arange(5) for p2 in np.arange(p1+1,5)]:
    print(i, j)

0 1
0 2
0 3
0 4
1 2
1 3
1 4
2 3
2 4
3 4


In [290]:
a = np.random.randint(0,5,(5,5))

In [308]:
np.random.uniform()

0.791202472008808

In [294]:
np.append(a[0][:2], a[1][2:])

array([4, 3, 0, 2, 3])

In [298]:
from joblib import Parallel, delayed
from math import sqrt
Parallel(n_jobs=1)(delayed(sqrt)(i**2) for i in range(10))

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

In [297]:
!pip install joblib